# LCA of Bitcoin Mining (consequential model)

In [1]:
import pandas as pd
import numpy as np
from lci_to_bw2 import *
from brightway2 import *
from matplotlib import pyplot as plt
import time
when = time.strftime("%Y%m%d")

In [2]:
projects.set_current('BTC_LCA')

In [3]:
projects

Brightway2 projects manager with 4 objects:
	BTC_Att_2019
	BTC_Att_LCA
	BTC_LCA
	default
Use `projects.report()` to get a report on all projects.

In [ ]:
#upload consequential ecoinvent v3.5 database (& change directory to find the file on your computer)
fpei34  = "/Users/susanne/Documents/O1a/Databases/Ecoinvent_conseq_3.5/datasets"
 
if 'ecoinvent 3.5 conseq' in databases:
     print("Database has already been imported")
else:
     ei34 = SingleOutputEcospold2Importer(fpei34, 'ecoinvent 3.5 conseq')
     ei34.apply_strategies()
     ei34.statistics()
 
ei34.write_database()

In [ ]:
bw2setup() # do this if its the first time only

In [4]:
databases

Databases dictionary with 6 object(s):
	BAU
	biosphere3
	ecoinvent 3.5 conseq
	efficiency
	location
	location&efficiency

## Data import

Upload Baseline Scenario (BAU) and creating a dictionary for it. 

In [5]:
BAU = pd.read_csv('BAU.csv', header = 0, sep = ",", encoding = 'utf-8-sig') 

# clean up 
BAU = BAU.drop(['Notes'], 1)  # remove the columns not needed
BAU['Exchange uncertainty type'] = BAU['Exchange uncertainty type'].fillna(0).astype(int) 

In [6]:
BAU_dict = lci_to_bw2(BAU)

In [7]:
# Write a bw2 database
databases
if 'BAU' in databases: del databases['BAU']
BAU = Database("BAU")
BAU.write(BAU_dict)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/08/2020 14:27:30
  Finished: 09/08/2020 14:27:30
  Total time elapsed: 00:00:00
  CPU %: 52.00
  Memory %: 0.95


Upload Scenario 1 (location-sensitive) and creating a dictionary for it. 

In [8]:
location = pd.read_csv('Scenario_1_location.csv', header = 0, sep = ",", encoding = 'utf-8-sig') 

# clean up 
location = location.drop(['Notes'], 1)  # remove the columns not needed
location['Exchange uncertainty type'] = location['Exchange uncertainty type'].fillna(0).astype(int) 

In [9]:
location_dict = lci_to_bw2(location)

In [10]:
# Write a bw2 database
databases
if 'location' in databases: del databases['location']
location = Database("location")
location.write(location_dict)

Writing activities to SQLite3 database:
0% [##################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/08/2020 14:27:35
  Finished: 09/08/2020 14:27:35
  Total time elapsed: 00:00:00
  CPU %: 62.80
  Memory %: 0.95


Upload Scenario 2 (equipment-sensitive) and creating a dictionary for it. 

In [14]:
efficiency = pd.read_csv('Scenario_2_efficiency.csv', header = 0, sep = ",", encoding = 'utf-8-sig') 

# clean up 
efficiency = efficiency.drop(['Notes'], 1)  # remove the columns not needed
efficiency['Exchange uncertainty type'] = efficiency['Exchange uncertainty type'].fillna(0).astype(int) 

In [15]:
efficiency_dict = lci_to_bw2(efficiency)

In [16]:
# Write a bw2 database
databases
if 'efficiency' in databases: del databases['efficiency']
efficiency = Database("efficiency")
efficiency.write(efficiency_dict)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/08/2020 14:27:50
  Finished: 09/08/2020 14:27:50
  Total time elapsed: 00:00:00
  CPU %: 35.60
  Memory %: 0.96


Upload Scenario 3 (increased efficiency and only new locations) and creating a dictionary for it. 

In [17]:
location_efficiency = pd.read_csv('Scenario_3_location&efficiency.csv', header = 0, sep = ",", encoding = 'utf-8-sig') 

# clean up 
location_efficiency = location_efficiency.drop(['Notes'], 1)  # remove the columns not needed
location_efficiency['Exchange uncertainty type'] = location_efficiency['Exchange uncertainty type'].fillna(0).astype(int) 

In [18]:
location_efficiency_dict = lci_to_bw2(location_efficiency)

In [19]:
# Write a bw2 database
databases
if 'location&efficiency' in databases: del databases['location&efficiency']
location_efficiency = Database("location&efficiency")
location_efficiency.write(location_efficiency_dict)

Writing activities to SQLite3 database:
0% [##################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/08/2020 14:27:53
  Finished: 09/08/2020 14:27:53
  Total time elapsed: 00:00:00
  CPU %: 77.70
  Memory %: 0.96


## Static LCA

In [20]:
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
mymethod

('IPCC 2013', 'climate change', 'GWP 100a')

In [21]:
acts_BAU = []
for activity in Database("BAU"):
        acts_BAU.append(activity['name'])
acts_BAU.sort()

acts_efficiency = []
for activity in Database("efficiency"):
        acts_efficiency.append(activity['name'])
acts_efficiency.sort()

acts_location = []
for activity in Database("location"):
        acts_location.append(activity['name'])
acts_location.sort()

acts_location_efficiency = []
for activity in Database("location&efficiency"):
        acts_location_efficiency.append(activity['name'])
acts_location_efficiency.sort()

In [22]:
myact_BAU = Database('BAU').get(acts_BAU[0])
myact_efficiency = Database('efficiency').get(acts_efficiency[0])
myact_location = Database('location').get(acts_location[0])
myact_location_efficiency = Database('location&efficiency').get(acts_location_efficiency[0])

In [23]:
FU_BAU = {myact_BAU: 1} 
lca_BAU = LCA(FU_BAU, mymethod)
lca_BAU.lci()
lca_BAU.lcia()

FU_efficiency = {myact_efficiency: 1} 
lca_efficiency = LCA(FU_efficiency, mymethod)
lca_efficiency.lci()
lca_efficiency.lcia()

FU_location = {myact_location: 1} 
lca_location = LCA(FU_location, mymethod)
lca_location.lci()
lca_location.lcia()

FU_location_efficiency = {myact_location_efficiency: 1} 
lca_location_efficiency = LCA(FU_location_efficiency, mymethod)
lca_location_efficiency.lci()
lca_location_efficiency.lcia()

print("The carbon footprints using IPCC 2013 for 1 TH are:")
print("{} kg CO2-eq for the baseline scenario (BAU)".format(lca_BAU.score))
print("{} kg CO2-eq for the 1st scenario (location)".format(lca_location.score))
print("{} kg CO2-eq for the 2nd scenario (efficiency)".format(lca_efficiency.score))
print("{} kg CO2-eq for the 3rd scenario (location & efficiency)".format(lca_location_efficiency.score))

The carbon footprints using IPCC 2013 for 1 TH are:
1.2396023369267433e-05 kg CO2-eq for the baseline scenario (BAU)
1.6185112903746263e-05 kg CO2-eq for the 1st scenario (location)
6.430004997169447e-06 kg CO2-eq for the 2nd scenario (efficiency)
8.395968372421257e-06 kg CO2-eq for the 3rd scenario (location & efficiency)
